In [18]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import os

import tensorflow
from keras.layers import Dense,Convolution2D,Flatten,MaxPooling2D,Reshape, Input, UpSampling2D,ZeroPadding2D,Dropout, BatchNormalization
from keras.models import Sequential,Model
from keras.utils import np_utils

In [19]:
dirs = os.listdir("/home/harshitbisht96/facial_recognition_cnn/data/celebs")


In [20]:
print (dirs)

['hrithik', 'rooney', 'monica', 'will']


In [21]:
folder_path="/home/harshitbisht96/facial_recognition_cnn/data/celebs/"

In [22]:
image_data=[]

In [23]:
labels=[]

In [24]:
label_dict={'hrithik':0,'monica':1,'rooney':2,'will':3}

In [25]:
from keras.preprocessing import image
from keras.optimizers import Adam

In [26]:
for ix in dirs:
    path=folder_path + ix + '/'
    images=os.listdir(path)
    for im in images:
        img=image.load_img(path+im,target_size=(224,224))
        img_array=image.img_to_array(img)
        image_data.append(img_array)
        labels.append(label_dict[ix])

In [27]:
import random

In [28]:
combined=list(zip(image_data,labels))
random.shuffle(combined)

image_data[:],labels[:]=zip(*combined)

In [29]:
X_train=np.array(image_data)
X_train.shape

Y_train=np.array(labels)
Y_train=np_utils.to_categorical(Y_train)


print (X_train.shape,Y_train.shape)

(121, 224, 224, 3) (121, 4)


In [30]:
inp=Input(shape=(224,224,3))

c1 = Convolution2D(220,(3,3),activation='relu',border_mode='same')(inp)
m0 = MaxPooling2D((2,2))(c1)
c2 = Convolution2D(100, (3,3), activation='relu',border_mode='same')(m0)
c3 = Convolution2D(50, (3,3), activation='relu',border_mode='same')(c2)
b0 = BatchNormalization()(c3)
m1 = MaxPooling2D((2,2))(b0)
c4 = Convolution2D(16, (3,3), activation='relu',border_mode='same')(m1)
c5 = Convolution2D(8,(3,3), activation='relu', border_mode='same')(c4)
b1 = BatchNormalization()(c5)
f1 = Flatten()(b1)
fc0 = Dense(220,activation='relu')(f1)

fc1 = Dense(64,activation='relu')(fc0)

fc2 = Dense(4,activation='softmax')(fc1)

model=Model(input=inp,output=fc2)
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(220, (3, 3), activation="relu", padding="same")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, (3, 3), activation="relu", padding="same")`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (3, 3), activation="relu", padding="same")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), activation="relu", padding="same")`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 224, 224, 220)     6160      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 112, 112, 220)     0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 112, 112, 100)     198100    
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 112, 112, 50)      45050     
_________________________________________________________________
batch_normalization_3 (Batch (None, 112, 112, 50)      200       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 56, 56, 50)        0         
__________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [31]:
adam=Adam(lr=0.00003)
model.compile(loss='binary_crossentropy',optimizer=adam,metrics=['accuracy'])

In [32]:
hist = model.fit(X_train,Y_train,
                    shuffle = True,
                    batch_size = 16,
                    epochs = 5,
                    validation_split=0.20
                    )

Train on 96 samples, validate on 25 samples
Epoch 1/5
96/96 [==============================] - 141s 1s/step - loss: 0.6804 - acc: 0.6979 - val_loss: 0.5895 - val_acc: 0.7200
Epoch 2/5
96/96 [==============================] - 99s 1s/step - loss: 0.3578 - acc: 0.8464 - val_loss: 0.5499 - val_acc: 0.7300
Epoch 3/5
96/96 [==============================] - 113s 1s/step - loss: 0.1977 - acc: 0.9635 - val_loss: 0.5440 - val_acc: 0.7600
Epoch 4/5
96/96 [==============================] - 113s 1s/step - loss: 0.1020 - acc: 0.9870 - val_loss: 0.5187 - val_acc: 0.7600
Epoch 5/5
96/96 [==============================] - 129s 1s/step - loss: 0.0537 - acc: 0.9974 - val_loss: 0.5134 - val_acc: 0.7500


In [33]:
# test_img=image.load_img("/home/harshitbisht96/ml-practise/test14.jpg",target_size=(224,224))
# test_img_array = image.img_to_array(test_img)
# tx = np.expand_dims(test_img_array,axis=0)
# plt.imshow(test_img)
# ty=model.predict(tx)
# label_dict = {v: k for k, v in label_dict.items()}
# print (label_dict[np.argmax(ty)])
label_dict = {v: k for k, v in label_dict.items()}
print (label_dict[0])

hrithik


In [34]:
test_img=image.load_img("/home/harshitbisht96/ml-practise/test15.jpg",target_size=(224,224))
test_img_array = image.img_to_array(test_img)
tx = np.expand_dims(test_img_array,axis=0)
plt.imshow(test_img)
ty=model.predict(tx)
index=np.argmax(ty)
print (label_dict[index])
    

FileNotFoundError: [Errno 2] No such file or directory: '/home/harshitbisht96/ml-practise/test15.jpg'

In [96]:
print (tx.shape)

(1, 224, 224, 3)
